### Show Deployed Images

In [3]:
!kubectl get deployments -n development

NAME          READY   UP-TO-DATE   AVAILABLE   AGE
main-mape     0/0     0            0           32d
postgres-db   1/1     1            1           32d
worker        0/0     0            0           5d16h


### Deploy Main

In [6]:
!kubectl scale deployment main-mape --replicas=0 --namespace=development

deployment.apps/main-mape scaled


### Change Confing

In [31]:
!curl -X POST http://127.0.0.1:8080/update_config \
     -d '{"expected_number_of_workers": 5, "namespace": "online-boutique"}' \
     -H "Content-Type: application/json"

curl: (7) Failed to connect to 127.0.0.1 port 8080 after 0 ms: Connection refused


### Deploy Workers

In [7]:
!kubectl scale deployment worker --replicas=0 --namespace=development

deployment.apps/worker scaled


### Show Results

In [34]:
import pandas as pd
import openpyxl
from openpyxl.chart import LineChart, Reference
from openpyxl.utils.dataframe import dataframe_to_rows
import psycopg2

def connect_to_db():
    conn = psycopg2.connect(
        dbname="mydatabase",
        user="user",
        password="password",
        host="localhost",
        port="5432"
    )
    return conn

conn = connect_to_db()
query = "SELECT * FROM stat_table"
df = pd.read_sql(query, conn)
conn.close()

df['timestamp'] = pd.to_datetime(df['timestamp'])

wb = openpyxl.Workbook()

for service_name, group in df.groupby('service'):
    group.reset_index(drop=True, inplace=True)
    ws = wb.create_sheet(title=service_name)
    
    for r_idx, row in enumerate(dataframe_to_rows(group, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx, column=c_idx, value=value)
    
    # Line chart for Total Risk
    chart1 = LineChart()
    data = Reference(ws, min_col=3, min_row=1, max_col=3, max_row=len(group)+1)
    times = Reference(ws, min_col=1, min_row=1, max_col=1, max_row=len(group)+1) # Timestamp
    chart1.add_data(data, titles_from_data=True)
    chart1.set_categories(times)
    chart1.title = "Total Risk Over Time"
    chart1.x_axis.title = "Time"
    chart1.y_axis.title = "Total Risk"
    chart1.width = 15  # width = 15 inches
    chart1.height = 10  # height = 10 inches
    ws.add_chart(chart1, "A20")
    
    # Line chart for CPU Usage
    chart2 = LineChart()
    data = Reference(ws, min_col=4, min_row=1, max_col=4, max_row=len(group)+1)
    chart2.add_data(data, titles_from_data=True)
    chart2.set_categories(times)
    chart2.title = "CPU Usage Over Time"
    chart2.x_axis.title = "Time"
    chart2.y_axis.title = "CPU Usage"
    chart2.width = 15
    chart2.height = 10
    ws.add_chart(chart2, "K20")
    
    # Line chart for Number of Replicas
    chart3 = LineChart()
    data = Reference(ws, min_col=5, min_row=1, max_col=5, max_row=len(group)+1)
    chart3.add_data(data, titles_from_data=True)
    chart3.set_categories(times)
    chart3.title = "Number of Replicas Over Time"
    chart3.x_axis.title = "Time"
    chart3.y_axis.title = "Number of Replicas"
    chart3.width = 15
    chart3.height = 10
    ws.add_chart(chart3, "W20")

del wb['Sheet']
wb.save("service_data.xlsx")


/tmp/ipykernel_785/1597273258.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


### Reset Tables

In [20]:
!kubectl scale deployment --all --replicas=1 -n google-shop
import psycopg2
import pandas as pd
import time
def connect_to_db():
    # Modify connection parameters according to your setup
    conn = psycopg2.connect(
        dbname="mydatabase",
        user="user",
        password="password",
        host="localhost",
        port="5432"
    )
    return conn
def truncate_actions_table():
    conn = connect_to_db()
    try:
        with conn.cursor() as cursor:
            cursor.execute("TRUNCATE TABLE actions_table RESTART IDENTITY CASCADE;")
            cursor.execute("TRUNCATE TABLE stat_table RESTART IDENTITY CASCADE;")
            conn.commit()
            print("Successfully truncated actions_table.")
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        print(traceback.format_exc())
        conn.rollback()
    finally:
        conn.close()

# Call this function whenever you need to empty the actions_table.
truncate_actions_table()
def reset_worker_cve_scan_status():
    conn = connect_to_db()
    try:
        with conn.cursor() as cursor:
            cursor.execute("UPDATE worker_cve_scan SET status='Unsolved';")
            conn.commit()
            print("Successfully updated status to 'Unsolved' for all records in worker_cve_scan.")
    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        print(traceback.format_exc())
        conn.rollback()
    finally:
        conn.close()

# Call this function whenever you need to reset the status.
reset_worker_cve_scan_status()

error: no objects passed to scale
Successfully truncated actions_table.
Successfully updated status to 'Unsolved' for all records in worker_cve_scan.
